In [46]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import time
import json
import gmaps
import os
from pprint import pprint
from scipy.stats import linregress
from config import api_key
from config import g_key

Store Part I results into DataFrame

Load the csv exported in Part I to a DataFrame

In [47]:
# Loan CSV file generated from WeatherPy Folder
weather_data_to_load = "../WeatherPy/output_data/cities.csv"
weather_data = pd.read_csv(weather_data_to_load)
weather_data.head(20)

,Unnamed: 0,Unnamed: 0.1,Unnamed: 0.1.1,Unnamed: 0.1.1.1,Unnamed: 0.1.1.1.1,Unnamed: 0.1.1.1.1.1,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,0,0,0,0,0,0,punta arenas,90,PH,1579714331,92,8.99,125.34,72.72,1.41
1,1,1,1,1,1,1,rikitea,0,PF,1579714332,80,-23.12,-134.97,78.06,12.53
2,2,2,2,2,2,2,atuona,33,PF,1579714332,75,-9.80,-139.03,80.82,13.38
3,3,3,3,3,3,3,skibbereen,75,IE,1579714108,87,51.55,-9.27,50.00,4.70
4,4,4,4,4,4,4,mwense,100,ZM,1579714332,98,-10.38,28.70,67.77,2.26
5,5,5,5,5,5,5,nikolskoye,75,RU,1579714333,61,59.70,30.79,32.00,15.66
6,6,6,6,6,6,6,avarua,100,CK,1579714333,88,-21.21,-159.78,80.60,11.41
7,7,7,7,7,7,7,jamestown,1,US,1579714333,62,42.10,-79.24,30.99,8.05
8,8,8,8,8,8,8,tuktoyaktuk,90,CA,1579714333,75,69.45,-133.04,-14.80,9.17
9,9,9,9,9,9,9,awbari,0,LY,1579714333,32,26.59,12.78,57.87,8.99


In [48]:
#cut unneeded columns
to_drop = ['Unnamed: 0','Unnamed: 0.1','Unnamed: 0.1.1','Unnamed: 0.1.1.1','Unnamed: 0.1.1.1.1', 'Unnamed: 0.1.1.1.1.1']
weather_data.drop(to_drop, inplace=True, axis=1)
weather_data

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,punta arenas,90,PH,1579714331,92,8.99,125.34,72.72,1.41
1,rikitea,0,PF,1579714332,80,-23.12,-134.97,78.06,12.53
2,atuona,33,PF,1579714332,75,-9.80,-139.03,80.82,13.38
3,skibbereen,75,IE,1579714108,87,51.55,-9.27,50.00,4.70
4,mwense,100,ZM,1579714332,98,-10.38,28.70,67.77,2.26
...,...,...,...,...,...,...,...,...,...
544,ambilobe,99,MG,1579715454,90,-13.20,49.05,78.62,9.78
545,tuatapere,54,NZ,1579715455,88,-46.13,167.68,57.00,11.99
546,itarema,93,BR,1579715317,59,-2.92,-39.92,88.02,10.60
547,nguru,0,NG,1579715455,16,12.88,10.45,74.28,12.37


 Humidity Heatmap

Configure gmaps.
Use the Lat and Lng as locations and Humidity as the weight.
Add Heatmap layer to map.

In [49]:
locations = weather_data[["Lat", "Lng"]].astype(float)
# locations

humidity = weather_data["Humidity"].astype(float)

# humidity

# figure_layout = {
#     'width' : '1000px',
#     'height':'400px',
#     'border':'1px solid black',
#     'padding':'1px',
#     'margin':'0 auto 0 auto'
# }

# fig = gmaps.figure(layout=figure_layout)

fig = gmaps.figure()

heatmap_layer = gmaps.heatmap_layer(locations, weights=humidity,
                                   dissipating=False, max_intensity=5,
                                   point_radius=1)

fig.add_layer(heatmap_layer)

fig

Figure(layout=FigureLayout(height='420px'))

 Create new DataFrame fitting weather criteria

Narrow down the cities to fit weather conditions.
Drop any rows will null values.

In [55]:
# Narrow down the DataFrame to find your ideal weather condition. For example:
# A max temperature lower than 80 degrees but higher than 70.
# Wind speed less than 10 mph.
# Zero cloudiness.
# Drop any rows that don't contain all three conditions. 
# You want to be sure the weather is ideal

new_types_df = pd.DataFrame(weather_data, columns = ["City", "Max Temp", "Wind Speed", "Cloudiness"])

max_temp = (new_types_df["Max Temp"] <= 80) & (new_types_df["Max Temp"] > 70)
wind_speed = new_types_df["Wind Speed"] < 10
cloudiness = new_types_df["Cloudiness"] == 0

# new_types_df[max_temp & wind_speed & cloudiness]
new_types_df[max_temp & wind_speed & cloudiness]

,City,Max Temp,Wind Speed,Cloudiness
156,latung,79.99,2.48,0
270,ruwi,73.40,9.17,0
368,guerrero negro,72.66,1.70,0
416,voh,74.91,6.02,0
463,lakatoro,79.09,9.33,0
523,acajutla,78.80,6.93,0


 Hotel Map

Store into variable named hotel_df.
Add a "Hotel Name" column to the DataFrame.
Set parameters to search for hotels with 5000 meters.
Hit the Google Places API for each city's coordinates.
Store the first Hotel result into the DataFrame.
Plot markers on top of the heatmap.

In [56]:
hotel_df = pd.DataFrame(weather_data, columns=["City", "Country", "Lat", "Lng"])

hotel_df["Hotel Name"] = ""
# hotel_df["Coordinates"]=""
hotel_df

,City,Country,Lat,Lng,Hotel Name
0,punta arenas,PH,8.99,125.34,
1,rikitea,PF,-23.12,-134.97,
2,atuona,PF,-9.80,-139.03,
3,skibbereen,IE,51.55,-9.27,
4,mwense,ZM,-10.38,28.70,
...,...,...,...,...,...
544,ambilobe,MG,-13.20,49.05,
545,tuatapere,NZ,-46.13,167.68,
546,itarema,BR,-2.92,-39.92,
547,nguru,NG,12.88,10.45,


In [59]:
hotellist = []

for i in range(len(hotel_df)):
    lat = hotel_df.loc[i]['Lat']
    lng = hotel_df.loc[i]['Lng']

    params = {
        "location": f"{lat},{lng}",
        "radius": 5000,
        "types" : "hotel",
        "key": g_key
    }
    
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
    requested = requests.get(base_url, params=params)
    jsn = requested.json()
    try:
        hotellist.append(jsn['results'][0]['name'])
    except:
        hotellist.append("")
hotel_df["Hotel Name"] = hotellist
hotel_df = hotel_df.dropna(how='any')
hotel_df.head()

,City,Country,Lat,Lng,Hotel Name
0,punta arenas,PH,8.99,125.34,Nasipit
1,rikitea,PF,-23.12,-134.97,Rikitea
2,atuona,PF,-9.80,-139.03,Atuona
3,skibbereen,IE,51.55,-9.27,Skibbereen
4,mwense,ZM,-10.38,28.70,Mwense


In [61]:
 # NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [62]:
# Add marker layer ontop of heat map

markers = gmaps.marker_layer(locations)
fig.add_layer(markers)
fig

# Display Map


Figure(layout=FigureLayout(height='420px'))